# Milestone 4
### Baseline Model
Aguillar, Goswami, Fallon

In this notebook, we are cleaning the data and getting it into a format where we can fit analytical models on it. I have saved the final output as a CSV to avoid long running times for future work on the dataset. It can be easily modified here.

ALERT: a lot of these cells can only be run once, running them twice without commenting things out may require you to start over. 

All variables created are np.number. The one exception is the loan issue date, which I have kept as datetime for the purposes of time series plotting.

The three other date variables have been converted to datetime and then a corresponding ordinal number.

In [11]:
import datetime as dt

import pandas as pd
pd.set_option('display.max_columns', 100)

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn import preprocessing
from sklearn.preprocessing import Imputer
from sklearn import tree
from sklearn import ensemble
from sklearn.cross_validation import KFold
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, export_graphviz
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import train_test_split
import StringIO
import matplotlib
import matplotlib.pyplot as plt
# special matplotlib argument for improved plots
from matplotlib import rcParams
from IPython.display import Image
import pydotplus
%matplotlib inline

from sklearn.feature_extraction import DictVectorizer

import glob

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

low_memory = False

In [12]:
files = glob.glob('/Users/Dwijo/Google Drive/CS 109/Homework/Final Project/Milestone 3/LC Data/LoanStats3[a-d].csv.zip')
files2 = glob.glob('/Users/Dwijo/Google Drive/CS 109/Homework/Final Project/Milestone 3/LC Data/LoanStats_2016Q[1-2].csv.zip')
files = files + files2

print 'IMPORT LOAN DATA\n'
print 'files being imported:\n' + str(files) +'\n'

#keep track of concatenation
i = 0

#start with the first file
data = pd.read_csv(files[0], header = 1, compression = 'zip')
### NOTE ADDITION OF CATEGORICAL YEAR COLUMN###

## There is actually an issue date variable, commenting this out
# data.loc[:, 'year'] = '2007-2011'

print 'file ' + str(i) + ' complete'

year_list = ['2012-2013', '2014', '2015', '2016', '2016']

#loop through remaining files
for file in files[1:len(files)]:
    data_cat = pd.read_csv(file, header = 1, compression = 'zip')
    ## There is actually an issue date variable, commenting this out
    #data_cat.loc[:, 'year'] = year_list[i]
    data = pd.concat((data, data_cat), axis = 0)
    
    #keep track of loading process
    i+=1
    print 'file ' + str(i) + ' complete'
    
    if i == len(files):
        print 'import complete'

#sense check shape, size etc. of dataset
size = np.shape(data)
variables = pd.DataFrame(data.columns.values, columns = ['Variables'])
data.head(n=5)

IMPORT LOAN DATA

files being imported:
['/Users/Dwijo/Google Drive/CS 109/Homework/Final Project/Milestone 3/LC Data/LoanStats3a.csv.zip', '/Users/Dwijo/Google Drive/CS 109/Homework/Final Project/Milestone 3/LC Data/LoanStats3b.csv.zip', '/Users/Dwijo/Google Drive/CS 109/Homework/Final Project/Milestone 3/LC Data/LoanStats3c.csv.zip', '/Users/Dwijo/Google Drive/CS 109/Homework/Final Project/Milestone 3/LC Data/LoanStats3d.csv.zip', '/Users/Dwijo/Google Drive/CS 109/Homework/Final Project/Milestone 3/LC Data/LoanStats_2016Q1.csv.zip', '/Users/Dwijo/Google Drive/CS 109/Homework/Final Project/Milestone 3/LC Data/LoanStats_2016Q2.csv.zip']

file 0 complete
file 1 complete
file 2 complete
file 3 complete
file 4 complete
file 5 complete


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,...,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,1077501,1296599.0,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,Dec-2011,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,Borrower added on 12/22/11 > I need to upgra...,credit_card,Computer,860xx,AZ,27.65,0.0,Jan-1985,1.0,NaN,NaN,3.0,0.0,13648.0,83.7%,9.0,f,0.00,0.00,5863.155187,5833.84,5000.00,863.16,0.00,0.00,0.00,Jan-2015,171.62,NaN,Sep-2016,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
1,1077430,1314167.0,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-2011,Charged Off,n,https://lendingclub.com/browse/loanDetail.acti...,Borrower added on 12/22/11 > I plan to use t...,car,bike,309xx,GA,1.00,0.0,Apr-1999,5.0,NaN,NaN,3.0,0.0,1687.0,9.4%,4.0,f,0.00,0.00,1008.710000,1008.71,456.46,435.17,0.00,117.08,1.11,Apr-2013,119.66,NaN,Sep-2016,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2,1077175,1313524.0,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,NaN,10+ years,RENT,12252.0,Not Verified,Dec-2011,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,small_business,real estate business,606xx,IL,8.72,0.0,Nov-2001,2.0,NaN,NaN,2.0,0.0,2956.0,98.5%,10.0,f,0.00,0.00,3005.666844,3005.67,2400.00,605.67,0.00,0.00,0.00,Jun-2014,649.91,NaN,Sep-2016,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
3,1076863,1277178.0,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200.0,Source Verified,Dec-2011,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,Borrower added on 12/21/11 > to pay for prop...,other,personel,917xx,CA,20.00,0.0,Feb-1996,1.0,35.0,NaN,10.0,0.0,5598.0,21%,37.0,f,0.00,0.00,12231.890000,12231.89,10000.00,2214.92,16.97,0.00,0.00,Jan-2015,357.48,NaN,Apr-2016,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
4,1075358,1311748.0,3000.0,3000.0,3000.0,60 months,12.69%,67.79,B,B5,University Medical Group,1 year,RENT,80000.0,Source Verified,Dec-2011,Current,n,ht

In [13]:
##########################################
### run cell only once to avoid errors ###
##########################################

#view variable names
variables

#isolate categorical variables
cat_list = data.select_dtypes(exclude = [np.number]).columns

#remove response variable from categorical list to deal with separately
response = ['loan_status']
cat_list = cat_list.drop('loan_status')

cat_list

,Variables
0,id
1,member_id
2,loan_amnt
3,funded_amnt
4,funded_amnt_inv
5,term
6,int_rate
7,installment
8,grade
9,sub_grade


Index([u'id', u'term', u'int_rate', u'grade', u'sub_grade', u'emp_title',
       u'emp_length', u'home_ownership', u'verification_status', u'issue_d',
       u'pymnt_plan', u'url', u'desc', u'purpose', u'title', u'zip_code',
       u'addr_state', u'earliest_cr_line', u'revol_util',
       u'initial_list_status', u'last_pymnt_d', u'next_pymnt_d',
       u'last_credit_pull_d', u'application_type',
       u'verification_status_joint'],
      dtype='object')

In [14]:
#review to see if any of these are number type variables in string format
data.ix[:,list(cat_list)].head(n=3)

,id,term,int_rate,grade,sub_grade,emp_title,emp_length,home_ownership,verification_status,issue_d,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,earliest_cr_line,revol_util,initial_list_status,last_pymnt_d,next_pymnt_d,last_credit_pull_d,application_type,verification_status_joint
0,1077501,36 months,10.65%,B,B2,NaN,10+ years,RENT,Verified,Dec-2011,n,https://lendingclub.com/browse/loanDetail.acti...,Borrower added on 12/22/11 > I need to upgra...,credit_card,Computer,860xx,AZ,Jan-1985,83.7%,f,Jan-2015,NaN,Sep-2016,INDIVIDUAL,NaN
1,1077430,60 months,15.27%,C,C4,Ryder,< 1 year,RENT,Source Verified,Dec-2011,n,https://lendingclub.com/browse/loanDetail.acti...,Borrower added on 12/22/11 > I plan to use t...,car,bike,309xx,GA,Apr-1999,9.4%,f,Apr-2013,NaN,Sep-2016,INDIVIDUAL,NaN
2,1077175,36 months,15.96%,C,C5,NaN,10+ years,RENT,Not Verified,Dec-2011,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,small_business,real estate business,606xx,IL,Nov-2001,98.5%,f,Jun-2014,NaN,Sep-2016,INDIVIDUAL,NaN


In [15]:
##########################################
### run cell only once to avoid errors ###
##########################################

#There are numeric-like terms and terms that we clearly don't want to dummify
## Note: keeping some variables with numbers as categorical

numdates_list = ['id', 'issue_d', 'int_rate', 'term', 'earliest_cr_line',
                 'last_pymnt_d', 'last_credit_pull_d', 'revol_util']

cat_list = cat_list.drop(numdates_list)


## drop from categorical list and overall dataframe ##
cat_list = cat_list.drop(['desc', 'url','next_pymnt_d', 'title',
                          'zip_code', 'emp_title', 'verification_status_joint'])

#I'm including the above drop list with id and member_id
data = data.drop(['id', 'member_id', 'desc', 'url','next_pymnt_d', 'title',
                  'zip_code', 'emp_title', 'verification_status_joint'], 1)

#### Justification for drops

First, extract variables that are not numerical but clearly numeric to deal with later.

Second, get rid of redundant or clearly uninformative variables:

ID seems useless for our purposes, index suffices as unique identifier. 
member_id seems uninformative, too linked to individual risk.
Drop zip code: state variable smaller dimensions and more likely to be helpful for generalizations.
Next payment date irrelevant.
Already have information on income, dropping emp_title.
Verification_status_joint is heavily NaN and there is already a verification_status variable.
Title does not seem like an informative variable and is too heterogeneous.

In [16]:
#review overall remaining variables
data.head(n=1)
np.shape(data)[1]

#review categorical list: I will make these 11 variables categorical
data.ix[:, list(cat_list)].head(n=1)
np.shape(data.ix[:, list(cat_list)])[1]

#review numerical and dates list: I will numerize/date these variables
data.ix[:, numdates_list].head(n=3)
np.shape(data.ix[:, numdates_list].head(n=1))[1]

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,...,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,10+ years,RENT,24000.0,Verified,Dec-2011,Fully Paid,n,credit_card,AZ,27.65,0.0,Jan-1985,1.0,NaN,NaN,3.0,0.0,13648.0,83.7%,9.0,f,0.0,0.0,5863.155187,5833.84,5000.0,863.16,0.0,0.0,0.0,Jan-2015,171.62,Sep-2016,0.0,NaN,1.0,INDIVIDUAL,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN


102

,grade,sub_grade,emp_length,home_ownership,verification_status,pymnt_plan,purpose,addr_state,initial_list_status,application_type
0,B,B2,10+ years,RENT,Verified,n,credit_card,AZ,f,INDIVIDUAL


10

,id,issue_d,int_rate,term,earliest_cr_line,last_pymnt_d,last_credit_pull_d,revol_util
0,NaN,Dec-2011,10.65%,36 months,Jan-1985,Jan-2015,Sep-2016,83.7%
1,NaN,Dec-2011,15.27%,60 months,Apr-1999,Apr-2013,Sep-2016,9.4%
2,NaN,Dec-2011,15.96%,36 months,Nov-2001,Jun-2014,Sep-2016,98.5%


8

In [17]:
######################################################
#### encode the categorical variables as dummies #####
######################################################

# create empty dataframe for categorical variables
data_cats = pd.DataFrame({})

# loop through the categorical variables and create categorical dataframe
for index, var in enumerate(cat_list):
    
    print 'Loop ' + str(index) + ':'
    print 'Categorical variable ' + str(var)
    #fit and transform
    cats = pd.get_dummies(data[var])
    
    #concatenate new categorical variables
    data_cats = pd.concat((data_cats, cats), axis = 1)
    
    print 'Categorical variable ' + str(index) + ' end dimensions:' + str(np.shape(data_cats))

Loop 0:
Categorical variable grade
Categorical variable 0 end dimensions:(1119194, 7)
Loop 1:
Categorical variable sub_grade
Categorical variable 1 end dimensions:(1119194, 42)
Loop 2:
Categorical variable emp_length
Categorical variable 2 end dimensions:(1119194, 54)
Loop 3:
Categorical variable home_ownership
Categorical variable 3 end dimensions:(1119194, 60)
Loop 4:
Categorical variable verification_status
Categorical variable 4 end dimensions:(1119194, 63)
Loop 5:
Categorical variable pymnt_plan
Categorical variable 5 end dimensions:(1119194, 65)
Loop 6:
Categorical variable purpose
Categorical variable 6 end dimensions:(1119194, 79)
Loop 7:
Categorical variable addr_state
Categorical variable 7 end dimensions:(1119194, 130)
Loop 8:
Categorical variable initial_list_status
Categorical variable 8 end dimensions:(1119194, 132)
Loop 9:
Categorical variable application_type
Categorical variable 9 end dimensions:(1119194, 135)


In [18]:
# check categorical variables before integrating into main dataframe

data_cats.head(n=3)

,A,B,C,D,E,F,G,A1,A2,A3,A4,A5,B1,B2,B3,B4,B5,C1,C2,C3,C4,C5,D1,D2,D3,D4,D5,E1,E2,E3,E4,E5,F1,F2,F3,F4,F5,G1,G2,G3,G4,G5,1 year,10+ years,2 years,3 years,4 years,5 years,6 years,7 years,...,CT,DC,DE,FL,GA,HI,IA,ID,IL,IN,KS,KY,LA,MA,MD,ME,MI,MN,MO,MS,MT,NC,ND,NE,NH,NJ,NM,NV,NY,OH,OK,OR,PA,RI,SC,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY,f,w,DIRECT_PAY,INDIVIDUAL,JOINT
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [19]:
########################################################
#### INTEGRATE CATEGORICAL VARIABLES INTO DATAFRAME ####
########################################################

#drop categorical variables
for index, var in enumerate(cat_list):
    print 'dropping variable: ' + str(var)
    data = data.drop(var, axis = 1)

#add in as dummies
data = pd.concat((data, data_cats), axis = 1)

dropping variable: grade
dropping variable: sub_grade
dropping variable: emp_length
dropping variable: home_ownership
dropping variable: verification_status
dropping variable: pymnt_plan
dropping variable: purpose
dropping variable: addr_state
dropping variable: initial_list_status
dropping variable: application_type


In [20]:
# check dataframe
data.head(n=3)

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,annual_inc,issue_d,loan_status,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,annual_inc_joint,dti_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,...,CT,DC,DE,FL,GA,HI,IA,ID,IL,IN,KS,KY,LA,MA,MD,ME,MI,MN,MO,MS,MT,NC,ND,NE,NH,NJ,NM,NV,NY,OH,OK,OR,PA,RI,SC,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY,f,w,DIRECT_PAY,INDIVIDUAL,JOINT
0,5000.0,5000.0,4975.0,36 months,10.65%,162.87,24000.0,Dec-2011,Fully Paid,27.65,0.0,Jan-1985,1.0,NaN,NaN,3.0,0.0,13648.0,83.7%,9.0,0.0,0.0,5863.155187,5833.84,5000.00,863.16,0.0,0.00,0.00,Jan-2015,171.62,Sep-2016,0.0,NaN,1.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,2500.0,2500.0,2500.0,60 months,15.27%,59.83,30000.0,Dec-2011,Charged Off,1.00,0.0,Apr-1999,5.0,NaN,NaN,3.0,0.0,1687.0,9.4%,4.0,0.0,0.0,1008.710000,1008.71,456.46,435.17,0.0,117.08,1.11,Apr-2013,119.66,Sep-2016,0.0,NaN,1.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,2400.0,2400.0,2400.0,36 months,15.96%,84.33,12252.0,Dec-2011,Fully Paid,8.72,0.0,Nov-2001,2.0,NaN,NaN,2.0,0.0,2956.0,98.5%,10.0,0.0,0.0,3005.666844,3005.67,2400.00,605.67,0.0,0.00,0.00,Jun-2014,649.91,Sep-2016,0.0,NaN,1.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [21]:
######################################################
#### make sure numerical variables are numerical #####
######################################################

#unfortunately, these have to be dealt with somewhat individually
numdates_list = ['issue_d', 'int_rate', 'term', 'earliest_cr_line',
                 'last_pymnt_d', 'last_credit_pull_d', 'revol_util']

In [22]:
### Convert issue_d to datetime

issue_d = data[numdates_list[0]]

issue_d = issue_d.apply(lambda x : pd.to_datetime(x, format = '%b-%Y'))
issue_d = issue_d.apply(lambda x : dt.datetime.toordinal(x))

data = data.drop(numdates_list[0], axis = 1)

#drop the old string dates, concatenate datetime for loan issue in front
data = pd.concat((issue_d, data), axis = 1 )

In [23]:
### Convert int_rate to float

int_rate = data[numdates_list[1]].apply(lambda x : str(x)[0:6])
int_rate = int_rate.apply(lambda x : float(x))
data[numdates_list[1]] = int_rate
data[numdates_list[1]].head(n=2)

0    10.65
1    15.27
Name: int_rate, dtype: float64

In [24]:
### Convert term in months to float

term = data[numdates_list[2]].apply(lambda x : str(x)[0:3])
term = term.apply(lambda x : float(x))
data[numdates_list[2]] = term
data[numdates_list[2]].head(n=2)

0    36.0
1    60.0
Name: term, dtype: float64

In [25]:
### For these date variables use datetime ordinal
other_dates = data[numdates_list[3:6]]

other_dates = other_dates.apply(lambda x : pd.to_datetime(x, format = '%b-%Y'))
other_dates.iloc[:, 0] = other_dates.iloc[:, 0].apply(lambda x : dt.datetime.toordinal(x))
other_dates.iloc[:, 1] = other_dates.iloc[:, 1].apply(lambda x : dt.datetime.toordinal(x))
other_dates.iloc[:, 2] = other_dates.iloc[:, 2].apply(lambda x : dt.datetime.toordinal(x))

data[numdates_list[3:6]] = other_dates
data[numdates_list[3:6]].head(n=2)

,earliest_cr_line,last_pymnt_d,last_credit_pull_d
0,724642,735599,736208
1,729845,734959,736208


In [26]:
### Convert revol_util to float

revol_util = data[numdates_list[6]]

revol_util = data[numdates_list[6]].apply(lambda x : str(x)[:-1] if str(x)[1].isdigit() else np.nan)        
revol_util = revol_util.apply(lambda x : float(x))

data[numdates_list[6]] = revol_util
data[numdates_list[6]].head(n=2)

0    83.7
1     NaN
Name: revol_util, dtype: float64

In [27]:
data.dtypes

issue_d                      int64
loan_amnt                  float64
funded_amnt                float64
funded_amnt_inv            float64
term                       float64
int_rate                   float64
installment                float64
annual_inc                 float64
loan_status                 object
dti                        float64
delinq_2yrs                float64
earliest_cr_line             int64
inq_last_6mths             float64
mths_since_last_delinq     float64
mths_since_last_record     float64
open_acc                   float64
pub_rec                    float64
revol_bal                  float64
revol_util                 float64
total_acc                  float64
out_prncp                  float64
out_prncp_inv              float64
total_pymnt                float64
total_pymnt_inv            float64
total_rec_prncp            float64
total_rec_int              float64
total_rec_late_fee         float64
recoveries                 float64
collection_recovery_

In [28]:
#put response variable last for clarity
col_list = data.columns.tolist()
col_list.remove('loan_status')
col_list.append('loan_status')
data = data[col_list]
data.head(n=2)
data.dtypes

# create a frame with a string response variable
data_rstr = data

,issue_d,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,annual_inc,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,annual_inc_joint,dti_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,...,DC,DE,FL,GA,HI,IA,ID,IL,IN,KS,KY,LA,MA,MD,ME,MI,MN,MO,MS,MT,NC,ND,NE,NH,NJ,NM,NV,NY,OH,OK,OR,PA,RI,SC,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY,f,w,DIRECT_PAY,INDIVIDUAL,JOINT,loan_status
0,734472,5000.0,5000.0,4975.0,36.0,10.65,162.87,24000.0,27.65,0.0,724642,1.0,NaN,NaN,3.0,0.0,13648.0,83.7,9.0,0.0,0.0,5863.155187,5833.84,5000.00,863.16,0.0,0.00,0.00,735599,171.62,736208,0.0,NaN,1.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,Fully Paid
1,734472,2500.0,2500.0,2500.0,60.0,15.27,59.83,30000.0,1.00,0.0,729845,5.0,NaN,NaN,3.0,0.0,1687.0,NaN,4.0,0.0,0.0,1008.710000,1008.71,456.46,435.17,0.0,117.08,1.11,734959,119.66,736208,0.0,NaN,1.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,Charged Off


issue_d                      int64
loan_amnt                  float64
funded_amnt                float64
funded_amnt_inv            float64
term                       float64
int_rate                   float64
installment                float64
annual_inc                 float64
dti                        float64
delinq_2yrs                float64
earliest_cr_line             int64
inq_last_6mths             float64
mths_since_last_delinq     float64
mths_since_last_record     float64
open_acc                   float64
pub_rec                    float64
revol_bal                  float64
revol_util                 float64
total_acc                  float64
out_prncp                  float64
out_prncp_inv              float64
total_pymnt                float64
total_pymnt_inv            float64
total_rec_prncp            float64
total_rec_int              float64
total_rec_late_fee         float64
recoveries                 float64
collection_recovery_fee    float64
last_pymnt_d        

In [29]:
#####
# Y #
#####

### Now make response variable loan_status binary

# 1 is default risk
# 0 is no default risk

loan_status = data['loan_status']
loan_status.head(n=2)

loan_status = loan_status.replace(to_replace = ["Fully Paid", 
                                      "Charged Off", 
                                      "Default", 
                                      "Late (31-120 days)", 
                                      "Current", 
                                      "In Grace Period",
                                      "Late (16-30 days)",
                                      "Does not meet the credit policy. Status:Fully Paid",
                                      "Does not meet the credit policy. Status:Charged Off"], 
                                      value = [0.0,
                                               1.0,
                                               1.0,
                                               1.0,
                                               0.0,
                                               0.0,
                                               1.0,
                                               0.0,
                                               1.0])

#loan_status = loan_status.apply(lambda x : float(x))

data['loan_status'] = loan_status

loan_status.head(n=2)

0     Fully Paid
1    Charged Off
Name: loan_status, dtype: object

0    0.0
1    1.0
Name: loan_status, dtype: float64

In [39]:
##creating list of numerical columns for imputing missing values##
numbers = data.select_dtypes(include=['float64', 'integer'], exclude=['object'])

##creating list of column names
num_name = numbers.columns.values

###trimming columns with too little data
missing_values = 0
missing_list = []
print "List of Columns with more than 30% Missing Values", "\n"

for name in num_name:
    count = data[name]
    missing_values = count.isnull().sum()
    if missing_values > (0.3*(1119194)):
        data[name].describe()
        print"Number of Missing Values:",missing_values
        print "\n"
        missing_list.append(name)

List of Columns with more than 30% Missing Values 



count    553539.000000
mean         33.983685
std          21.905406
min           0.000000
25%                NaN
50%                NaN
75%                NaN
max         192.000000
Name: mths_since_last_delinq, dtype: float64

Number of Missing Values: 565655




count    179105.000000
mean         69.247045
std          27.336729
min           0.000000
25%                NaN
50%                NaN
75%                NaN
max         129.000000
Name: mths_since_last_record, dtype: float64

Number of Missing Values: 940089




...

Number of Missing Values: 866143




count    266121.000000
mean         39.673615
std          22.755968
min           0.000000
25%                NaN
50%                NaN
75%                NaN
max         195.000000
Name: mths_since_recent_bc_dlq, dtype: float64

Number of Missing Values: 853073




count    373898.000000
mean         35.657468
std          22.436949
min           0.000000
25%                NaN
50%                NaN
75%                NaN
max         197.000000
Name: mths_since_recent_revol_delinq, dtype: float64

Number of Missing Values: 745296




In [21]:
##replacing missing values with mean values
for n in num_name:
    data[n] = data[n].fillna(value=data[n].mean())

In [23]:
#Review final dataframe

data.head(n=5)
np.shape(data)

,issue_d,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,annual_inc,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,annual_inc_joint,dti_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,...,DC,DE,FL,GA,HI,IA,ID,IL,IN,KS,KY,LA,MA,MD,ME,MI,MN,MO,MS,MT,NC,ND,NE,NH,NJ,NM,NV,NY,OH,OK,OR,PA,RI,SC,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY,f,w,DIRECT_PAY,INDIVIDUAL,JOINT,loan_status
0,734472,5000.0,5000.0,4975.0,36.0,10.65,162.87,24000.0,27.65,0.0,724642,1.0,33.983685,69.247045,3.0,0.0,13648.0,83.700000,9.0,0.00,0.00,5863.155187,5833.84,5000.00,863.16,0.00,0.00,0.00,735599,171.62,736208,0.0,44.336923,1.0,108683.595881,18.497685,0.0,233.945249,140631.725624,1.02194,2.819764,0.748771,1.627559,21.613877,35453.205586,70.91569,1.372082,2.920617,5948.75622,60.344269,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1,734472,2500.0,2500.0,2500.0,60.0,15.27,59.83,30000.0,1.00,0.0,729845,5.0,33.983685,69.247045,3.0,0.0,1687.0,56.089109,4.0,0.00,0.00,1008.710000,1008.71,456.46,435.17,0.00,117.08,1.11,734959,119.66,736208,0.0,44.336923,1.0,108683.595881,18.497685,0.0,233.945249,140631.725624,1.02194,2.819764,0.748771,1.627559,21.613877,35453.205586,70.91569,1.372082,2.920617,5948.75622,60.344269,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
2,734472,2400.0,2400.0,2400.0,36.0,15.96,84.33,12252.0,8.72,0.0,730790,2.0,33.983685,69.247045,2.0,0.0,2956.0,98.500000,10.0,0.00,0.00,3005.666844,3005.67,2400.00,605.67,0.00,0.00,0.00,735385,649.91,736208,0.0,44.336923,1.0,108683.595881,18.497685,0.0,233.945249,140631.725624,1.02194,2.819764,0.748771,1.627559,21.613877,35453.205586,70.91569,1.372082,2.920617,5948.75622,60.344269,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
3,734472,10000.0,10000.0,10000.0,36.0,13.49,339.31,49200.0,20.00,0.0,728690,1.0,35.000000,69.247045,10.0,0.0,5598.0,21.000000,37.0,0.00,0.00,12231.890000,12231.89,10000.00,2214.92,16.97,0.00,0.00,735599,357.48,736055,0.0,44.336923,1.0,108683.595881,18.497685,0.0,233.945249,140631.725624,1.02194,2.819764,0.748771,1.627559,21.613877,35453.205586,70.91569,1.372082,2.920617,5948.75622,60.344269,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,734472,3000.0,3000.0,3000.0,60.0,12.69,67.79,80000.0,17.94,0.0,728659,0.0,38.000000,69.247045,15.0,0.0,27783.0,53.900000,38.0,270.78,270.78,3784.490000,3784.49,2729.22,1055.27,0.00,0.00,0.00,736208,67.79,736208,0.0,44.336923,1.0,108683.595881,18.497685,0.0,233.945249,140631.725624,1.02194,2.819764,0.748771,1.627559,21.613877,35453.205586,70.91569,1.372082,2.920617,5948.75622,60.344269,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


(1119194, 227)

In [24]:
# Save a file with the original response variables in string format in last row
#    This way we can quickly change our approach to the response variable

#%timeit data_rstr.to_csv('/Users/James/Documents/Fall 2016 (2)/Data Science/Final project/LC Data/LC_cd_rstr.gzip', compression = 'gzip', chunksize = 1000000)

In [25]:
# Save the main file

#%timeit data.to_csv('/Users/James/Documents/Fall 2016 (2)/Data Science/Final project/LC Data/LC_cd.gzip', compression = 'gzip', chunksize = 1000000)

In [41]:
# Split label and instances
y = data.iloc[:, -1]
x = data.iloc[:, :-1].as_matrix()

#reshaping y because otherwise the classifer doesn't work
y=np.asarray(y, dtype="|S6")

# # dictionary vectorizing x in order to use strings
# vec = DictVectorizer()
# X = vec.fit_transform([item[0] for item in x]).as_matrix(columns=None)

print 'predictors shape:', x.shape
print 'outcome shape:', y.shape

predictors shape: (1119194, 226)
outcome shape: (1119194,)


In [45]:
y_sum = np.sum(y)

TypeError: cannot perform reduce with flexible type

In [55]:
# Parameters for tuning
n_trees = np.arange(10, 50, 20)  # Trees and depth are explored on an exponentially growing space,
depths = np.arange(2, 5)   # since it is assumed that trees and depth will add accuracy in a decaying fashion.

# To keep track of the best model
best_score = 0

# Run grid search for model with 5-fold cross validation
print '3-fold cross validation:'

for trees in n_trees:
    for depth in depths:
        
        # Cross validation for every experiment
        k_folds = KFold(x.shape[0], n_folds=3, shuffle=True)
        scores = []
        for train_indices, validation_indices in k_folds:
            # Generate training data
            x_train_cv = x[train_indices]
            y_train_cv = y[train_indices]
            # Generate validation data
            x_validate = x[validation_indices]
            y_validate = y[validation_indices]
            
            # Fit random forest on training data
            model = ensemble.RandomForestClassifier(n_estimators=trees, max_depth=depth)
            model.fit(x_train_cv, y_train_cv)
            # Score on validation data
            scores += [model.score(x_validate, y_validate)]
        
        # Record and report accuracy
        average_score = np.mean(scores)
        print "Trees:", trees, "Depth:", depth, "Score:", average_score
        
        # Update our record of the best parameters see so far
        if average_score > best_score:
            best_score = average_score
            best_trees = trees
            best_depth = depth

3-fold cross validation:


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=2, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=2, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=2, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

Trees: 10 Depth: 2 Score: 0.905329192398


...

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=4, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=30, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=4, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=30, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=4, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=30, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

Trees: 30 Depth: 4 Score: 0.93492460754


NameError: name 'x_train_no_census' is not defined

In [57]:
# Fit model on entire dataser using optimized number of trees and depth

print 'Chosen number of trees, depth:', best_trees, ',', best_depth
print 'Test accuracy:', model.score(x, y)

 Chosen number of trees, depth: 10 , 4
Test accuracy: 0.939309896229


In [ ]:
# # Print decision tree model 'model', already fitted
# def display_dt(model):
#     dummy_io = StringIO.StringIO() 
#     tree.export_graphviz(model, out_file = dummy_io) 
#     print dummy_io.getvalue()

# def print_tree(estimator, features, class_names=None, filled=True):
#     tree = estimator
#     names = features
#     color = filled
#     classn = class_names
    
#     dot_data = StringIO.StringIO()
#     export_graphviz(estimator, out_file=dot_data, feature_names=features, proportion=True, class_names=classn, filled=filled)
#     graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
#     return(graph)

In [ ]:
# import StringIO

# display_dt(loan)

In [ ]:
# #dropping outcome column to create list of predictor features
# data_features = data.drop(['loan_status'])

# features = data.columns.values

# #creating graph for decision tree
# graph = print_tree(loan, features=features, class_names=['No', 'Yes'])
# Image(graph.create_png())